# TP 4

### Par Mathieu Marquis, Charles-Antoine Parent et Thomas Piuze

In [ ]:
import numpy as np
import graphinglib as gl

## 4.1 : Intégration numérique de la dynamique épidémiologique SIS

### **a)** Identifiez les deux états stationnaires $0 \leq i^*_{1,2} \leq 1$ pour lesquels $\frac{di}{d\tau} = 0$, de même que les conditions sur $R_0$ pour lesquelles ces états stationnaires sont possibles (s’il y a lieu).

À partir de l'équation suivante:
$$ \frac{di}{d\tau} = (R_0-1)i-R_0i^2, $$
nous pouvons identifier les deux états stationnaires:
$$ \begin{align*}
    0 &= (R_0-1)i-R_0i^2 \\
    0 &= i(R_0-1-R_0i) \\
    i &= \begin{cases}
            0 \\
            \frac{R_0-1}{R_0}
        \end{cases}
    \end{align*} $$
Un premier état stationnaire se produit pour $i^*_1=0$, ce qui est logique, car en l'absence d'individus infectés, la maladie ne peut se propager. Un deuxième cas, moins intuitif, se produit pour $i^*_2=\frac{R_0-1}{R_0}$. Le premier cas n'implique aucune condition sur $R_0$. En revanche, le deuxième cas restreint les valeurs possibles de $R_0$, car nous devons avoir $0 \leq i^*_2 \leq 1$. Puisque $R_0=\beta/\alpha$, et que $\alpha,\beta>0$, $R_0$ est toujours positif. Nous avons donc:
$$ \begin{align*}
        0 &\leq \frac{R_0-1}{R_0} \leq 1 \\
        0 &\leq R_0-1 \leq R_0 \\
        1 &\leq R_0 \leq R_0+1
    \end{align*} $$
La condition $R_0 \leq R_0+1$ est redondante. Cependant, la condition $R_0 \geq 1$ est plus restrictive, car elle nécessite que $\beta \geq \alpha$.

### **b)** Obtenez une solution analytique $i(\tau)$ (valide pour $i \geq 0$). Vous noterez $i(0) = i_0$. Indice : Il s’agit d’une EDO de Bernouilli.

L'EDO de Bernouilli à résoudre est:
$$ \frac{di}{d\tau} = (R_0-1)i-R_0i^2 $$
Multiplions cette équation par $-1/i^2$:
$$ -\frac{1}{i^2}\frac{di}{d\tau} = \frac{(1-R_0)}{i}+R_0 $$ 
Posons ensuite le changement de variable $u=\frac{1}{i}$, de telle sorte que:
$$ u = \frac{1}{i} \longrightarrow \frac{du}{d\tau} = \frac{du}{di}\frac{di}{d\tau} = -\frac{1}{i^2}\frac{di}{d\tau} $$
L'équation à résoudre devient alors:
$$ \frac{du}{d\tau} = (1-R_0)u+R_0 $$
Soit une équation différentielle ordinaire linéaire du premier ordre. La forme générale d'une telle équation est:
$$ y'(x)+p(x)y(x) = q(x) $$
Dont la solution est donnée par:
$$ y(x) = e^{-\int p(x)dx}\left(\int e^{\int p(x)dx}q(x)dx+C\right) $$
Où $C$ est une constante déterminée par la condition initiale. Dans notre cas, $x=\tau$, $y(x)=u(\tau)$, $y'(x)=\frac{du(\tau)}{d\tau}$, $p(x)=(R_0-1)$ et $q(x)=R_0$. La solution est donc:
$$ \begin{align*}
        u(\tau) &= e^{-\int (R_0-1)d\tau}\left(\int e^{\int (R_0-1)d\tau}R_0d\tau+C\right) \\
        &= e^{-(R_0-1)\tau}\left(\int R_0e^{(R_0-1)\tau}d\tau+C\right) \\
        &= e^{-(R_0-1)\tau}\left(\frac{R_0}{R_0-1}e^{(R_0-1)\tau}+C\right) \\
        &= Ce^{-(R_0-1)\tau}+\frac{R_0}{R_0-1}
    \end{align*} $$
Puisque $i(0)=i_0$, $u(0)=\frac{1}{i(0)}=\frac{1}{i_0}$. Par conséquent:
$$ u(0) = C+\frac{R_0}{R_0-1} = \frac{1}{i_0} \longrightarrow C = \frac{1}{i_0}-\frac{R_0}{R_0-1} $$
Donc:
$$ \begin{align*}
        u(\tau) &= \left(\frac{1}{i_0}-\frac{R_0}{R_0-1}\right)e^{-(R_0-1)\tau}+\frac{R_0}{R_0-1} \\
        i(\tau) &= \frac{1}{i(\tau)} \\
        i(\tau) &= \frac{1}{\left(\frac{1}{i_0}-\frac{R_0}{R_0-1}\right)e^{-(R_0-1)\tau}+\frac{R_0}{R_0-1}} \\
        i(\tau) &= \frac{i_0(R_0-1)}{\left(R_0(1-i_0)-1\right)e^{-(R_0-1)\tau}+R_0i_0}
    \end{align*} $$
Soit la solution recherchée.

### **c)** Implémentez les intégrateurs d’Euler, de Runge-Kutta d’ordre 2 et de Runge-Kutta d’ordre 4. Vos fonctions devront retourner les trajectoires composées des fractions de la population infectée $\{i_s\}_{s=0,...,T}$ et des temps $\{\tau_s\}_{s=0,...,T}$ auxquels ces fractions correspondent. La valeur de $T$ sera fixée ci-dessous.

### **d)** Implémentez la méthode du ratio dorée (ou un autre optimisateur de votre choix) qui vous permettra de trouver la longueur de pas $h$ permettant d’obtenir une erreur sur la trajectoire numérique $\varepsilon(h)$ bornée par $(1\pm0.01)\delta$, où $\delta$ sera spécifiée ci-dessous. L’erreur sera calculée selon
$$ \varepsilon(h) = \sqrt{\frac{1}{T+1}\sum_{s=0}^T(i_s-i(\tau_s))^2}, $$
### où $i(\cdot)$ est la solution analytique obtenue en **b**. N’hésitez pas à considérer une méthode euristique pour bien cerner l’intervalle de départ pour $h$ fourni à la méthode du ratio doré. À l’aide de votre algorithme, identifiez une valeur de pas d’intégration $h$ permettant de limiter $\varepsilon(h)$ à l’intervalle $[0.99\delta,1.01\delta]$ pour plusieurs valeurs de $\delta \in [10^{-9},10^{-6}]$, pour plusieurs couples $(i_0,R_0)$ et pour chacun des trois intégrateurs implémentés en **c**.

### **e)** À l’aide des résultats obtenus en **d**, obtenez numériquement l’ordre $d$ de l’erreur globale $\varepsilon(h) \propto h^d$ effectuée par chacun des trois intégrateurs implémentés en **c**. Il est possible d’obtenir $d$ via une régression (linéaire) en observant que $\log \varepsilon(h) \propto d \log h$.

## 4.2 : Simulation Monte-Carlo de la dynamique SIS

### **a)** Déterminez l’expression qui gouverne la probabilité de générer un graphe de l’ensemble $G(n,p)$ qui contiendra $m$ liens. Détaillez votre raisonnement pour obtenir cette expression.

### **b)** Déterminez également le nombre de liens moyens que possède un graphe issu du modèle $G(n,p)$ ainsi que l’écart-type sur le nombre de liens. Détaillez votre raisonnement pour obtenir ces expressions. Si des propriétés sont utilisées, elles doivent être dûment citées et/ou démontrées également.

### **c)** Implémentez un algorithme qui génère des graphes en utilisant le modèle d’Erdős–Rényi avec les paramètres $n = 100$ et $p = 0.05$. Une fois programmé, obtenez empiriquement la moyenne, l’écart-type et la distribution du nombre de liens dans les réseaux issus de votre algorithme. Comparez ces résultats aux valeurs obtenues en **a** et **b**. (Indice : un graphe est parfois simplement représenté par sa matrice d’adjacence. Vous pouvez utiliser cette représentation ou une autre structure de votre choix qui permet d’encoder l’information pertinente.)

### **d)** Implémentez la simulation de la dynamique SIS sur réseau en utilisant le module `NetworkX`. Une fois votre algorithme fonctionnel, générez les courbes moyennes du nombre de nœuds infectés et susceptibles en fonction du temps pour le réseau fourni (*reseau.adj*). Pour ouvrir ce réseau, utilisez la fonction `read_adjlist()` de `NetworkX`. Pour vos simulations, utilisez $\alpha = 0.05$, $\beta = 0.1$, une proportion initiale de nœuds infectés de 10% et 100 pas de temps. Comparez les résultats de la simulation avec le modèle étudié dans la première partie du travail et discutez.